In [1]:
import pandas as pd 
from datetime import datetime, date, time, timedelta

 
df = pd.read_csv("crime.csv") 

In [2]:
df['DATE'], df['TIME'] = df['FIRST_OCCURRENCE_DATE'].str.split(' ', 1).str


df['DATE'] = pd.to_datetime(df['DATE'])
df['day_of_week'] = df['DATE'].dt.day_name()

In [3]:
df_violent = df[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID','GEO_LON', 'GEO_LAT']]
df_violent = df_violent.loc[(df_violent.OFFENSE_CATEGORY_ID == 'aggravated-assult')|(df_violent.OFFENSE_CATEGORY_ID == 'sexual-assult')|(df_violent.OFFENSE_CATEGORY_ID == 'robbery')|(df_violent.OFFENSE_CATEGORY_ID == 'murder')].copy()
df_violent_am =  df_violent.loc[df_violent.TIME.str[-2:] == "AM" ].copy()
df_violent_am['TIME'] = [x[:-3] for x in df_violent_am['TIME']]
df_violent_am['TIME'] =  pd.to_datetime(df_violent_am['TIME'], format='%H:%M:%S')


In [4]:
df_violent = df_violent[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID','GEO_LON','GEO_LAT']]
df_violent = df_violent.loc[(df_violent.OFFENSE_CATEGORY_ID == 'aggravated-assult')|(df_violent.OFFENSE_CATEGORY_ID == 'sexual-assult')|(df_violent.OFFENSE_CATEGORY_ID == 'robbery')|(df_violent.OFFENSE_CATEGORY_ID == 'murder')].copy()
df_violent_pm =  df_violent.loc[df_violent.TIME.str[-2:] == "PM" ].copy()
df_violent_pm['TIME'] = [x[:-3] for x in df_violent_pm['TIME']]
df_violent_pm['TIME'] =  pd.to_datetime(df_violent_pm['TIME'], format='%H:%M:%S')
df_violent_pm['TIME'] = df_violent_pm['TIME'] + timedelta(hours=12)

In [5]:
final_violent = pd.concat([df_violent_am, df_violent_pm], ignore_index=True, sort =False)

block1 = '09/19/18 04:00:00'
block2 = '09/19/18 08:00:00'
block3 = '09/19/18 12:00:00'
block4 = '09/19/18 16:00:00'
block5 = '09/19/18 20:00:00'
block6 = '09/19/18 23:59:00'
datetime_object1 = datetime.strptime(block1, '%m/%d/%y %H:%M:%S')
datetime_object2 = datetime.strptime(block2, '%m/%d/%y %H:%M:%S')
datetime_object3 = datetime.strptime(block3, '%m/%d/%y %H:%M:%S')
datetime_object4 = datetime.strptime(block4, '%m/%d/%y %H:%M:%S')
datetime_object5 = datetime.strptime(block5, '%m/%d/%y %H:%M:%S')
datetime_object6 = datetime.strptime(block6, '%m/%d/%y %H:%M:%S')


In [6]:
final_violent.loc[final_violent['TIME'].dt.time <= datetime_object1.time(), 'TIME'] = datetime_object1

final_violent.loc[(final_violent['TIME'].dt.time > datetime_object1.time())&(final_violent['TIME'].dt.time <= datetime_object2.time()), 'TIME'] = datetime_object2

final_violent.loc[(final_violent['TIME'].dt.time > datetime_object2.time())&(final_violent['TIME'].dt.time <= datetime_object3.time()), 'TIME'] = datetime_object3

final_violent.loc[(final_violent['TIME'].dt.time > datetime_object3.time())&(final_violent['TIME'].dt.time <= datetime_object4.time()), 'TIME'] = datetime_object4

final_violent.loc[(final_violent['TIME'].dt.time > datetime_object4.time())&(final_violent['TIME'].dt.time <= datetime_object5.time()), 'TIME'] = datetime_object5

final_violent.loc[(final_violent['TIME'].dt.time > datetime_object5.time())&(final_violent['TIME'].dt.time <= datetime_object6.time()), 'TIME'] = datetime_object6



final_violent['TIME'] = [d.time() for d in final_violent['TIME']]
final_violent

,OFFENSE_CATEGORY_ID,TIME,day_of_week,NEIGHBORHOOD_ID,GEO_LON,GEO_LAT
0,robbery,12:00:00,Tuesday,belcaro,-104.958432,39.698094
1,robbery,04:00:00,Monday,valverde,-105.020006,39.717271
2,robbery,16:00:00,Friday,cbd,-104.993294,39.742580
3,robbery,04:00:00,Friday,five-points,-104.988641,39.754559
4,robbery,08:00:00,Tuesday,sun-valley,-105.024629,39.733243
...,...,...,...,...,...,...
7152,robbery,20:00:00,Thursday,capitol-hill,-104.975104,39.739859
7153,robbery,23:59:00,Thursday,athmar-park,-105.021884,39.710844
7154,robbery,20:00:00,Thursday,ruby-hill,-105.017990,39.691026
7155,robbery,23:59:00,Sunday,gateway-green-valley-ranch,-104.746999,39.782084


In [10]:
violent_csv = final_violent[['day_of_week','NEIGHBORHOOD_ID', 'TIME']]
violent_plot_csv = final_violent[['day_of_week','NEIGHBORHOOD_ID', 'TIME','GEO_LON', 'GEO_LAT']]
violent_csv = violent_csv.sample(5000)
violent_csv.to_csv('Violent.csv')
violent_plot_csv.to_csv('Violent_Plot.csv')


!python apriori.py -f 'Violent.csv' -s 0.003

item: ('Saturday', 'north-capitol-hill') , 0.003
item: ('23:59:00', 'sun-valley') , 0.003
item: ('speer', '23:59:00') , 0.003
item: ('20:00:00', 'civic-center') , 0.003
item: ('23:59:00', 'washington-virginia-vale') , 0.003
item: ('westwood', 'Wednesday') , 0.003
item: ('Monday', 'northeast-park-hill') , 0.003
item: ('Monday', 'baker') , 0.003
item: ('west-colfax', 'Tuesday') , 0.003
item: ('Wednesday', 'lincoln-park') , 0.003
item: ('north-capitol-hill', 'Tuesday') , 0.003
item: ('20:00:00', 'hampden') , 0.003
item: ('harvey-park', '23:59:00') , 0.003
item: ('20:00:00', 'athmar-park') , 0.003
item: ('20:00:00', 'washington-virginia-vale') , 0.003
item: ('23:59:00', 'barnum') , 0.003
item: ('Sunday', 'hampden') , 0.003
item: ('16:00:00', 'hampden') , 0.003
item: ('montbello', '12:00:00') , 0.003
item: ('five-points', 'Sunday', '04:00:00') , 0.003
item: ('west-colfax', 'Friday') , 0.003
item: ('23:59:00', 'highland') , 0.003
item: ('Saturday', 'civic-center') , 0.003
item: ('16:00:00', 